# STEP 0. Import

In [1]:
import os
import json
import zipfile
import urllib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

# STEP 1. Load Dataset


In [2]:
url = 'https://storage.googleapis.com/download.tensorflow.org/data/sarcasm.json'
urllib.request.urlretrieve(url, 'sarcasm.json')

('sarcasm.json', <http.client.HTTPMessage at 0x16b86c62490>)

`datas` 3개  출력
* `article_link`: 뉴스 기사 URL
* `headline`: 뉴스기사의 제목
* `is_sarcastic`: 비꼬는 기사 여부 (비꼼: 1, 일반: 0)

In [3]:
with open('sarcasm.json') as f:
    datas = json.load(f)
datas[:3]

[{'article_link': 'https://www.huffingtonpost.com/entry/versace-black-code_us_5861fbefe4b0de3a08f600d5',
  'headline': "former versace store clerk sues over secret 'black code' for minority shoppers",
  'is_sarcastic': 0},
 {'article_link': 'https://www.huffingtonpost.com/entry/roseanne-revival-review_us_5ab3a497e4b054d118e04365',
  'headline': "the 'roseanne' revival catches up to our thorny political mood, for better and worse",
  'is_sarcastic': 0},
 {'article_link': 'https://local.theonion.com/mom-starting-to-fear-son-s-web-series-closest-thing-she-1819576697',
  'headline': "mom starting to fear son's web series closest thing she will have to grandchild",
  'is_sarcastic': 1}]

# STEP 2. 전처리 (sentences, labels)
* X (Feature): sentences
* Y (Label): label

In [4]:
# 코드를 실행하세요.
# 비어있는 리스트를 가지는 sentences와 labels 변수를 생성하세요.
sentences = []
labels = []

In [6]:
# 반복문을 이용하여 datas의 데이터를 하나씩 data에 대입받으세요.
  # 대입받은 data에서 키가 headline인 value를 sentences에 저장하세요.
  # 대입받은 data에서 키가 is_sarcastic인 value를 labels에 저장하세요.
for i in range(len(datas)):
    sentences.append(datas[i]['headline'])
    labels.append(datas[i]['is_sarcastic'])

In [7]:
# 슬라이싱을 이용하여 sentences에 저장된 5개를 조회하세요
sentences[:5]

["former versace store clerk sues over secret 'black code' for minority shoppers",
 "the 'roseanne' revival catches up to our thorny political mood, for better and worse",
 "mom starting to fear son's web series closest thing she will have to grandchild",
 'boehner just wants wife to listen, not come up with alternative debt-reduction ideas',
 'j.k. rowling wishes snape happy birthday in the most magical way']

In [8]:
# 슬라이싱을 이용하여 labels에 저장된 5개를 조회하세요
labels[:5]

[0, 0, 1, 1, 0]

In [9]:
# 슬라이싱을 이용하여 처음부터 20000까지를 train에 저장하세요
training_size = 20000
train_sentences = sentences[:training_size]
train_labels = labels[:training_size]

In [10]:
# 슬라이싱을 이용하여 20000부터 마지막까지를 test에 저장하세요
validation_sentences = sentences[training_size:]
validation_labels = labels[training_size:]

단어의 토큰화를 진행합니다.
* `num_words`: 단어 max 사이즈를 지정합니다. 가장 **빈도수가 높은** 단어부터 저장합니다.
* `oov_token`: 단어 토큰에 없는 단어를 어떻게 표기할 것인지 지정해줍니다.

In [11]:
# 코드를 실행하세요.
# vocab_size는 1000 oov_tok는 "<OOV>"라는 값을 가지도록 변수를 생성하세요.
vocab_size = 1000
oov_tok = "<OOV>"

In [12]:
# Tokenizer를 생성하세요. num_words는 vocab_size를 oov_token는 oov_tok를 지정하세요.
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)

In [13]:
# 생성된 tokenizer의 fit_on_texts 함수를 이용하여 train_sentences를 피팅시키세요.
tokenizer.fit_on_texts(train_sentences)

In [14]:
# 코드를 실행하세요.
# 사전 확인
for key, value in tokenizer.word_index.items():
    print(f'{key}  \t======>\t {value}')
    if value == 25:
        break

<OOV>  	======>	 1
to  	======>	 2
of  	======>	 3
the  	======>	 4
in  	======>	 5
for  	======>	 6
a  	======>	 7
on  	======>	 8
and  	======>	 9
with  	======>	 10
is  	======>	 11
new  	======>	 12
trump  	======>	 13
man  	======>	 14
from  	======>	 15
at  	======>	 16
about  	======>	 17
you  	======>	 18
by  	======>	 19
this  	======>	 20
after  	======>	 21
be  	======>	 22
up  	======>	 23
out  	======>	 24
that  	======>	 25


In [15]:
# 토큰의 길이를 확인하세요.
  # len 함수 이용
len(tokenizer.word_index.items())

25637

In [16]:
# 피팅된 tokenizer의 texts_to_sequences 함수를 이용하여 데이터를 토큰화 시키세요.
train_sequences = tokenizer.texts_to_sequences(train_sentences)
validation_sequences = tokenizer.texts_to_sequences(validation_sentences)

In [17]:
# 코드를 실행하세요.
# 토큰화 결과 확인
train_sequences[:5]
# 빈도수로 지정한 num_words=1000 에 의거하여, 빈도수가 1000번째보다 떨어지는 단어는 자동으로 1로 치환

[[328, 1, 799, 1, 1, 47, 389, 1, 1, 6, 1, 1],
 [4, 1, 1, 1, 23, 2, 161, 1, 390, 1, 6, 251, 9, 889],
 [153, 890, 2, 891, 1, 1, 595, 1, 221, 133, 36, 45, 2, 1],
 [1, 38, 213, 382, 2, 1, 29, 288, 23, 10, 1, 1, 1, 958],
 [715, 672, 1, 1, 1, 662, 553, 5, 4, 92, 1, 90]]

In [18]:
# train_sequences에서 각 문장의 단어 개수의 최대값과, validation_sequences에서 각 문장의 단어 개수의 최대값을 구하세요.
max(len(review) for review in train_sequences),max(len(review) for review in validation_sequences)

(40, 38)

In [19]:
## 그대로 실행하세요.
# 한 문장의 최대 단어 숫자를 의미하는 max_length 변수를 만들고 40을 저장하세요.
max_length = 40

# 잘라낼 문장의 위치를 의미하는 trunc_type 변수를 만들고 'post'를 저장하세요.
trunc_type='post'

# 채워줄 문장의 위치를 의미하는 padding_type 변수를 만들고 'post'를 저장하세요.
padding_type='post'

In [20]:
# pad_sequences 함수를 이용하여 train_sequences과 validation_sequences에 패딩을 적용하세요.
 # maxlen는 max_length를, truncating는 trunc_type을, padding은 padding_type을 적용하세요.
train_padded = pad_sequences(train_sequences, maxlen=max_length, truncating=trunc_type, padding=padding_type)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, truncating=trunc_type, padding=padding_type)

In [21]:
# 패딩이 완료된 데이터의 shape을 조회하세요.
train_padded.shape, validation_padded.shape

((20000, 40), (6709, 40))

In [31]:
# train_padded의 상위 3개의 데이터를 조회하세요.
train_padded[:3]

array([[328,   1, 799,   1,   1,  47, 389,   1,   1,   6,   1,   1,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0],
       [  4,   1,   1,   1,  23,   2, 161,   1, 390,   1,   6, 251,   9,
        889,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0],
       [153, 890,   2, 891,   1,   1, 595,   1, 221, 133,  36,  45,   2,
          1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0]])

In [24]:
# train_labels과 validation_labels을 numpy array로 변경하세요.
train_labels = np.array(train_labels)
validation_labels = np.array(validation_labels)

# STEP 3. 모델 정의 (Sequential)

In [28]:
## 모델을 정의하세요. (Embedding Layer ==> RNN(LSTM, GRU) Layer ==>  Dense Layer)
# Embedding Layer의 output size는 16으로 지정하세요.
# Embedding Layer의 input_length는 max_length로 지정하세요.
model = Sequential()
model.add(Embedding(input_dim=1000,output_dim=16, input_length=max_length))
model.add(LSTM(64))
model.add(Dense(32))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 40, 16)            16000     
                                                                 
 lstm_1 (LSTM)               (None, 64)                20736     
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 38,849
Trainable params: 38,849
Non-trainable params: 0
_________________________________________________________________


# STEP 4. 컴파일 (compile)

In [29]:
# adam과 binary_crossentropy를 사용하여 모델을 컴파일하세요.
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

## ModelCheckpoint: 체크포인트 생성

In [30]:
# 코드를 실행하세요.
# ModelCheckpoint를 생성하세요.
checkpoint = ModelCheckpoint('model.h5',
                             save_best_only=True, 
                             monitor='val_loss',
                             verbose=1)

In [35]:
# 데이터를 학습하세요,
model.fit(train_padded, train_labels, 
          callbacks=checkpoint,epochs=15, 
          verbose=1, validation_split=0.2)

Epoch 1/15
499/500 [============================>.] - ETA: 0s - loss: 0.5678 - acc: 0.6776
Epoch 1: val_loss improved from inf to 0.41572, saving model to model.h5
500/500 [==============================] - 12s 20ms/step - loss: 0.5674 - acc: 0.6780 - val_loss: 0.4157 - val_acc: 0.8105
Epoch 2/15
498/500 [============================>.] - ETA: 0s - loss: 0.3913 - acc: 0.8231
Epoch 2: val_loss improved from 0.41572 to 0.38996, saving model to model.h5
500/500 [==============================] - 9s 18ms/step - loss: 0.3913 - acc: 0.8231 - val_loss: 0.3900 - val_acc: 0.8257
Epoch 3/15
498/500 [============================>.] - ETA: 0s - loss: 0.3740 - acc: 0.8324
Epoch 3: val_loss did not improve from 0.38996
500/500 [==============================] - 9s 18ms/step - loss: 0.3740 - acc: 0.8324 - val_loss: 0.3923 - val_acc: 0.8265
Epoch 4/15
499/500 [============================>.] - ETA: 0s - loss: 0.3577 - acc: 0.8377
Epoch 4: val_loss improved from 0.38996 to 0.38795, saving model to mode

In [37]:
# load_model 함수로 저장된 모델을 불러오세요
loaded_model = load_model('./model.h5')

In [38]:
# evaluate 함수를 이용하여 모델의 성능을 평가하세요
loaded_model.evaluate(validation_padded, validation_labels)

210/210 [==============================] - 2s 8ms/step - loss: 0.4027 - acc: 0.8131


[0.402656227350235, 0.8130869269371033]